In [82]:
import json
import re
from tqdm import tqdm
import networkx as nx
from collections import deque
from gradio_client import Client
import re
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial
import concurrent.futures

In [63]:
dataset_path = "data/commonsenseqa_validation.json"

with open(dataset_path, "r") as fp:
    csqa_dataset = json.load(fp)

len(csqa_dataset)

1221

In [64]:
client = Client("cstr/conceptnet_normalized")

relations = [
            'RelatedTo','IsA','PartOf','HasA','UsedFor','CapableOf','AtLocation',
            'Causes','HasSubevent','HasFirstSubevent','HasLastSubevent',
            'HasPrerequisite','HasProperty','MotivatedByGoal','ObstructedBy',
            'Desires','CreatedBy','Synonym','Antonym','DistinctFrom','DerivedFrom',
            'SymbolOf','DefinedAs','MannerOf','LocatedNear','HasContext','SimilarTo',
            'EtymologicallyRelatedTo','EtymologicallyDerivedFrom','CausesDesire',
            'MadeOf','ReceivesAction','ExternalURL','NotDesires','NotUsedFor',
            'NotCapableOf','NotHasProperty'
        ]

Loaded as API: https://cstr-conceptnet-normalized.hf.space ✔


In [65]:
def get_conceptnet_profile(word, relations):
    result = client.predict(
        word=word,
        lang="en",
        selected_relations=relations,
        api_name="/get_semantic_profile"
    )
    return result

In [66]:
def parse_conceptnet_profile(profile_text):
    """
    Parse ConceptNet semantic profile text into a dictionary.
    
    Args:
        profile_text: String output from get_conceptnet_profile
        
    Returns:
        Dictionary with relation types as keys and list of (word, score) tuples as values.
        Only includes relations that have values, sorted by score in descending order.
    """
    result = {}
    
    # Extract the queried word from the header
    # Pattern: # 🧠 Semantic Profile: 'word' (EN)
    header_match = re.search(r"# 🧠 Semantic Profile: '([^']+)'", profile_text)
    queried_word = header_match.group(1) if header_match else None
    
    # Normalize the queried word for comparison (both space and underscore versions)
    if queried_word:
        queried_normalized_space = queried_word.replace('_', ' ')
        queried_normalized_underscore = queried_word.replace(' ', '_')
    else:
        queried_normalized_space = None
        queried_normalized_underscore = None
    
    # Split by relation headers (## RelationType)
    sections = re.split(r'## (\w+)', profile_text)
    
    # sections[0] is the header before first relation, then alternates between relation name and content
    for i in range(1, len(sections), 2):
        relation = sections[i]
        content = sections[i + 1] if i + 1 < len(sections) else ""
        
        # Temporary list for this relation
        relation_list = []
        
        # Find all relation entries
        # Pattern: - *word1* or **word1** RelationType → *word2* or **word2** `[score]`
        pattern = r'-\s+(?:\*\*?([^*]+?)\*\*?)\s+\w+\s+→\s+(?:\*\*?([^*]+?)\*\*?)\s+`\[([0-9.]+)\]`'
        matches = re.findall(pattern, content)
        
        for match in matches:
            word1, word2, score = match
            # Remove any extra whitespace
            word1 = word1.strip()
            word2 = word2.strip()
            score = float(score)
            
            # Check if either word matches the queried word (in either format)
            def is_queried_word(word):
                if not queried_word:
                    return False
                word_space = word.replace('_', ' ')
                word_underscore = word.replace(' ', '_')
                return (word == queried_word or 
                        word_space == queried_normalized_space or 
                        word_underscore == queried_normalized_underscore)
            
            # Determine which word is NOT the queried word
            if is_queried_word(word1):
                target_word = word2
            elif is_queried_word(word2):
                target_word = word1
            else:
                # If neither matches exactly, prefer word1 (usually the related concept)
                target_word = word1
            
            relation_list.append((target_word, score))
        
        # Only add to result if there are values, and sort by score descending
        if relation_list:
            # Sort by score (second element of tuple) in descending order
            relation_list.sort(key=lambda x: x[1], reverse=True)
            result[relation] = relation_list
    
    return result

In [67]:
conceptnet_text = get_conceptnet_profile("revolving door", relations)
conceptnet_data = parse_conceptnet_profile(conceptnet_text)
conceptnet_data

{'RelatedTo': [('drehtür', 1.0),
  ('revolving doors', 1.0),
  ('tourniquet', 1.0),
  ('bussola', 1.0)],
 'UsedFor': [('entering building', 3.464),
  ('enter building', 1.0),
  ('exiting building', 1.0),
  ('getting into building', 1.0)],
 'AtLocation': [('lobby', 2.0),
  ('bank', 1.0),
  ('building', 1.0),
  ('department store', 1.0),
  ('entrance to building', 1.0),
  ('hotel lobby', 1.0),
  ('mall', 1.0)],
 'Synonym': [('drehtür', 1.0)]}

Concepts and question concept

In [11]:
concepts = []
for data_point in tqdm(csqa_dataset):
    choices = data_point['choices']['text']
    question_concept = data_point['question_concept']

    temp_concept = choices + [question_concept]
    concepts.extend(temp_concept)

concepts = list(set(concepts))
print(f"Total unique concepts to fetch: {len(concepts)}")

100%|██████████| 1221/1221 [00:00<00:00, 187708.29it/s]

Total unique concepts to fetch: 4010


In [ ]:
conceptnet_relations = {}
for concept in tqdm(concepts):
    conceptnet_text = get_conceptnet_profile(concept, relations)
    conceptnet_data = parse_conceptnet_profile(conceptnet_text)
    conceptnet_relations[concept] = conceptnet_data


In [81]:
def fetch_conceptnet_data(concept, relations):
    """Helper function to fetch and parse ConceptNet data for a single concept"""
    try:
        conceptnet_text = get_conceptnet_profile(concept, relations)
        conceptnet_data = parse_conceptnet_profile(conceptnet_text)
        return concept, conceptnet_data
    except Exception as e:
        print(f"Error processing {concept}: {e}")
        return concept, {}


In [ ]:

# Use ThreadPoolExecutor for parallel processing
conceptnet_relations = {}
max_workers = 5  # Adjust this number based on your needs and API limits

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Create a partial function with fixed relations parameter
    fetch_func = partial(fetch_conceptnet_data, relations=relations)
    
    # Submit all tasks
    future_to_concept = {executor.submit(fetch_func, concept): concept for concept in concepts}
    
    # Process completed tasks with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_concept), total=len(concepts)):
        concept, data = future.result()
        conceptnet_relations[concept] = data

  0%|          | 20/4010 [00:10<27:54,  2.38it/s] 

Error processing buying products: 


  3%|▎         | 101/4010 [00:47<28:37,  2.28it/s]

Error processing book: 


 19%|█▉        | 774/4010 [05:31<18:39,  2.89it/s]

...

 91%|█████████▏| 3667/4010 [26:07<02:10,  2.64it/s]

Error processing taking tests: 
Error processing being employed: 


 94%|█████████▎| 3753/4010 [26:43<01:39,  2.59it/s]

Error processing sell products: 
Error processing bench: 


100%|██████████| 4010/4010 [28:33<00:00,  2.34it/s]


In [ ]:
# # Store the data as needed, e.g., save to a file or database
# with open(f"data/conceptnet_relations.json", "w") as fp:
#     json.dump(conceptnet_relations, fp)

In [ ]:
# with open(f"data/conceptnet_relations.json", "r") as fp:
#     conceptnet_relations = json.load(fp)

In [61]:
len(conceptnet_relations)

4010

In [25]:
concept = "being employed"
# conceptnet_relations[concept]
fetch_conceptnet_data(concept, relations)

('being employed',
 {'Causes': [('applying for job', 1.0), ('getting contract', 1.0)]})

In [54]:
concept = "buying products"
conceptnet_relations[concept] = fetch_conceptnet_data(concept, relations)[1]
conceptnet_relations[concept]

{'UsedFor': [('consumption', 2.0),
  ('anticipated enjoyment', 1.0),
  ('being consumer', 1.0),
  ('filling house', 1.0),
  ('getting food', 1.0),
  ('getting things', 1.0),
  ('having things', 1.0),
  ('shopping', 1.0),
  ('suburban shopping mall', 1.0)],
 'Causes': [('spending money', 4.0),
  ('spend money', 3.464),
  ('being short on money', 2.0),
  ("activating country's economy", 1.0),
  ('agony', 1.0),
  ('bankruptcy', 1.0),
  ('being able to use', 1.0),
  ('seeing exhibits', 1.0)],
 'HasSubevent': [('compare prices', 2.0),
  ('attending sale', 1.0),
  ('being enticed', 1.0),
  ('being outbid', 1.0),
  ('break', 1.0),
  ('carrying to car', 1.0),
  ('check price', 1.0)],
 'HasPrerequisite': [('money', 4.472),
  ('cash', 1.0),
  ('desire', 1.0),
  ('shopping', 1.0),
  ('shopping list', 1.0),
  ('time', 1.0),
  ('woman and credit card', 1.0)]}

('bench',
 {'RelatedTo': [('bench press', 2.0),
   ('تخت', 1.0),
   ('صفة', 1.0),
   ('مصطبة', 1.0),
   ('نضد', 1.0),
   ('bank', 1.0),
   ('tisch', 1.0),
   ('sitting', 0.529),
   ('parks', 0.418),
   ('chair', 0.376),
   ('sit', 0.335),
   ('seat', 0.287),
   ('sit on', 0.218),
   ('place', 0.171)],
  'IsA': [('long seat with no backrest', 2.0), ('pew', 2.0), ('sit on', 1.0)],
  'UsedFor': [('catching breath', 1.0),
   ('growing plants', 1.0),
   ('legislating', 1.0),
   ('lying down', 1.0),
   ('observing others', 1.0),
   ('resting', 1.0),
   ('sit and rest', 1.0)],
  'AtLocation': [('bus stop', 4.0),
   ('bus depot', 3.464),
   ('train station', 2.828),
   ('state park', 2.0),
   ('dugout', 1.0),
   ('garden', 1.0),
   ('lawn', 1.0)],
  'Synonym': [('bank', 1.0),
   ('sitzbank', 1.0),
   ('bench press', 1.0),
   ('banc', 1.0),
   ('établi', 1.0)],
  'DistinctFrom': [('chair', 0.142)],
  'DerivedFrom': [('bench press', 1.0),
   ('bencher', 1.0),
   ('alebench', 1.0),
   ('backbench

concepts of keywords from question

In [70]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [72]:
def extract_keywords_spacy(question):
    doc = nlp(question)

    keywords = []
    for token in doc:
        # Filter out stopwords, punctuation, and select meaningful POS
        if not token.is_stop and not token.is_punct:
            if token.pos_ in {"NOUN", "PROPN", "VERB", "ADJ"}:
                keywords.append(token.lemma_.lower())

    return list(dict.fromkeys(keywords))  # Remove duplicates, preserve order


# Example
question = "Why would someone bring an umbrella outside on a sunny day?"
print(extract_keywords_spacy(question))

['bring', 'umbrella', 'sunny', 'day']


In [73]:
def extract_keywords_spacy(question):
    doc = nlp(question)
    return [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop
        and not token.is_punct
        and token.pos_ in {"NOUN", "PROPN", "VERB", "ADJ"}
    ]

def extract_keywords_batch(questions, n_process=4):
    results = []
    for doc in nlp.pipe(questions, n_process=n_process, batch_size=32):
        keywords = [
            token.lemma_.lower()
            for token in doc
            if not token.is_stop
            and not token.is_punct
            and token.pos_ in {"NOUN", "PROPN", "VERB", "ADJ"}
        ]
        results.append(keywords)
    return results


# Example usage:
questions = [
    "Why would someone bring an umbrella outside on a sunny day?",
    "Where do people usually keep their bicycles?",
    "Why do cats like to sit in warm places?"
]

print(extract_keywords_batch(questions, n_process=4))

[['bring', 'umbrella', 'sunny', 'day'], ['people', 'bicycle'], ['cat', 'like', 'sit', 'warm', 'place']]


In [74]:
question = "Why would someone bring an umbrella outside on a sunny day?"
print(extract_keywords_spacy(question))

['bring', 'umbrella', 'sunny', 'day']


In [76]:
questions = [data_point['question'] for data_point in csqa_dataset]

print(f"Total questions: {len(questions)}")

Total questions: 1221


In [77]:
question_keywords = []
total_keywords = []
for question in tqdm(questions):
    keywords = extract_keywords_spacy(question)
    question_keywords.append(keywords)
    total_keywords.extend(keywords)


100%|██████████| 1221/1221 [00:15<00:00, 80.08it/s]


In [79]:
len(total_keywords)
total_keywords = list(set(total_keywords))
print(f"Total unique keywords to fetch: {len(total_keywords)}")

Total unique keywords to fetch: 2026


In [83]:
# Use ThreadPoolExecutor for parallel processing
conceptnet_relations = {}
max_workers = 5  # Adjust this number based on your needs and API limits

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Create a partial function with fixed relations parameter
    fetch_func = partial(fetch_conceptnet_data, relations=relations)
    
    # Submit all tasks
    future_to_concept = {executor.submit(fetch_func, concept): concept for concept in total_keywords}
    
    # Process completed tasks with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_concept), total=len(total_keywords)):
        concept, data = future.result()
        conceptnet_relations[concept] = data

 15%|█▍        | 301/2026 [02:05<11:53,  2.42it/s]

Error processing expedite: 


 23%|██▎       | 474/2026 [03:15<10:19,  2.51it/s]

Error processing cash: 


100%|██████████| 2026/2026 [13:17<00:00,  2.54it/s]


In [85]:
concept = "cash"
conceptnet_relations[concept] = fetch_conceptnet_data(concept, relations)[1]
conceptnet_relations[concept]

{'RelatedTo': [('money', 5.338),
  ('payment', 1.856),
  ('money', 1.511),
  ('نض', 1.0),
  ('نضض', 1.0),
  ('نقد', 1.0),
  ('نقدي', 1.0),
  ('نقود', 1.0),
  ('green', 0.291),
  ('johnny', 0.181),
  ('johnny precedes', 0.181),
  ('precedes', 0.181),
  ('bills', 0.176),
  ('dollar', 0.176)],
 'IsA': [('another word for money', 6.0),
  ('money', 3.464),
  ('another name for paper money', 2.0),
  ('colloquialism for legal tender', 1.0),
  ('liquid form of money', 1.0),
  ('money in species i.e', 1.0),
  ('paper money', 1.0),
  ('another way to say money', 1.0),
  ('one kind of money', 1.0)],
 'UsedFor': [('buy things with', 2.828),
  ('bribe', 2.0),
  ('buy food or clothes', 1.0),
  ('buy things', 1.0),
  ('buying gas', 1.0),
  ('buying items', 1.0),
  ('buying things', 1.0),
  ('dollar bill', 1.0)],
 'AtLocation': [('bank', 2.0),
  ('purse', 2.0),
  ('cash register', 1.0),
  ('checkstand', 1.0)],
 'HasPrerequisite': [('buying', 1.0),
  ('buying christmas presents', 1.0),
  ('buying fresh

In [87]:
with open(f"data/conceptnet_relations.json", "r") as fp:
    data = json.load(fp)

In [88]:
final_data = {**data, **conceptnet_relations}
len(final_data)

5230

In [ ]:
# with open(f"data/conceptnet_relations.json", "w") as fp:
#     json.dump(final_data, fp)

In [90]:
concept = "scissors"
fetch_conceptnet_data(concept, relations)[1]

{'RelatedTo': [('cut', 4.984),
  ('paper', 1.197),
  ('مقص', 1.0),
  ('schere', 1.0),
  ('binary noun', 1.0),
  ('chisel', 1.0),
  ('cut', 1.0),
  ('cutting', 0.921),
  ('blades', 0.33),
  ('two', 0.33),
  ('two blades', 0.33),
  ('cutting tool', 0.265),
  ('tool', 0.265),
  ('paper', 0.181)],
 'IsA': [('clippers', 1.0), ('safety scissors', 1.0)],
 'UsedFor': [('cut', 5.657),
  ('construction paper', 1.0),
  ('cut hair', 1.0),
  ('cut leather', 1.0),
  ('cut paper or cloth', 1.0),
  ('cut paper snowflakes', 1.0),
  ('cut ribbons', 1.0)],
 'CapableOf': [('cut paper', 5.292),
  ('become blunt', 1.0),
  ('cut cardboard', 1.0),
  ('part paper', 1.0),
  ('tear cloth', 1.0),
  ('tear paper', 1.0)],
 'AtLocation': [('desk', 3.464),
  ('cabinet', 2.0),
  ('backpack', 1.0),
  ('drawer', 1.0),
  ('house', 1.0)],
 'HasPrerequisite': [('going to get haircut', 2.828),
  ('cutting hair', 2.0),
  ('having haircut', 2.0)],
 'HasProperty': [('sharp', 2.0), ('useful for cutting things', 1.0)],
 'Synonym